In [ ]:
import numpy as np 
from np.testing import assert_allclose
from sklearn.utils import shuffle

import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from keras.utils import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.optimizers import Adam
from tensorflow import keras
from tensorflow.keras.layers import Dense,BatchNormalization, Flatten, Dropout
from tensorflow.keras.models import Model, load_model, Sequential
from keras.applications.vgg16 import VGG16
from tensorflow.keras.callbacks import ModelCheckpoint


In [ ]:
# mount drive if colab is used
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# print(tf.__version__)

2.9.2


In [ ]:
img_size = (256,256)

TRAIN_DIR = './drive/MyDrive/brainmri/archive/Training'
TEST_DIR = './drive/MyDrive/brainmri/archive/Testing'


dataset_train = image_dataset_from_directory(directory = TRAIN_DIR, 
                                           label_mode = 'categorical', 
                                           shuffle = False, 
                                           batch_size = None,
                                           image_size = img_size,
                                           crop_to_aspect_ratio = True)

dataset_test = image_dataset_from_directory(directory = TEST_DIR, 
                                           label_mode = 'categorical', 
                                           shuffle = False, 
                                           batch_size = None,
                                           image_size = img_size,
                                           crop_to_aspect_ratio = True)

# need to normalize the data
def normalize(image,label):
    image = tf.cast(image/255. ,tf.float32)
    label = tf.cast(label ,tf.float32)
    return image,label

class_names = dataset_train.class_names
num_classes = len(class_names)



dataset_train = dataset_train.map(normalize)
dataset_test = dataset_test.map(normalize)


          

Found 2870 files belonging to 4 classes.
Found 394 files belonging to 4 classes.


In [ ]:
x_train = []
y_train = []
for (image, label) in dataset_train:
    x_train.append(image)
    y_train.append(label)
    
x_train = np.asarray(x_train)
y_train = np.asarray(y_train)
y_train = y_train.astype(np.uint8)

# shuffle the dataset
x_train, y_train = shuffle(x_train, y_train, random_state = 100)

print(x_train.shape)

Define function visualize_data to plot a grid of training images with respective label.

In [ ]:
def visualize_data(images, categories, class_names):
    
    L = 4
    W = 4

    fig, axes = plt.subplots(L, W, figsize = (10, 10))
    axes = axes.ravel()

    for i in range(L*W):
        j = np.random.randint(1, images.shape[0])
        axes[i].imshow(images[j], cmap = 'gray')
        axes[i].set_title(class_names[np.argmax(y_train[j])])
        axes[i].axis('off')
        plt.show()


visualize_data(x_train, y_train, class_names)    

Use image augmentation on the training set.

---



In [ ]:
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range = 0.1,
    width_shift_range=0.05,
    height_shift_range=0.05,
    validation_split=0.25)

datagen.fit(x_train)

Load the VGG16 model and add some layers.

In [ ]:
basemodel=VGG16(weights='imagenet', include_top=False,input_shape=(img_size[0],img_size[1],3))

for layer in basemodel.layers[:-2]:
  layer.trainable = False

headmodel = basemodel.output

headmodel = Flatten(name = 'flatten')(headmodel)
headmodel = BatchNormalization()(headmodel)
headmodel = Dropout(0.5)(headmodel)

headmodel = Dense(1024, activation = 'relu')(headmodel)
headmodel = Dropout(0.5)(headmodel)

headmodel = Dense(1024, activation = 'relu')(headmodel)
headmodel = Dropout(0.5)(headmodel)

headmodel = Dense(1024, activation = 'relu')(headmodel)
headmodel = Dropout(0.4)(headmodel) 

# output for the 4 labels
headmodel = Dense(4, activation = 'softmax')(headmodel) 
model = Model(inputs = basemodel.input, outputs = headmodel)

In [ ]:
model.summary()

In [ ]:
optimizer = Adam(learning_rate=1e-3)
model.compile(loss = 'categorical_crossentropy', optimizer = optimizer, metrics= ["accuracy"])

Fit the model.

In [ ]:
filepath = "./drive/MyDrive/brainmri/model_VGG16.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks = [checkpoint]
epochs = 80
batch_size=32

history = model.fit(datagen.flow(x_train, y_train, batch_size=batch_size, subset='training'), 
                    validation_data=datagen.flow(x_train, y_train,batch_size=batch_size, subset='validation'),
                    epochs=epochs, verbose=1,
                    callbacks = callbacks)


In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy'] 
loss = history.history['loss']
val_loss = history.history['val_loss'] 

Plot: train and validation accuracy.

In [ ]:
epochs = range(len(acc))
fig = plt.figure(figsize = (14,7))
plt.plot(epochs, acc,  label = 'Training Accuracy')
plt.plot(epochs, val_acc,  label = 'Validation Accuracy')
plt.legend(loc = 'upper left')

plt.savefig('./drive/MyDrive/brainmri/accuracy_model_VGG16.jpg')
plt.show()
plt.close()

In [ ]:
epochs = range(len(acc))
fig = plt.figure(figsize = (14,7))
plt.plot(epochs, loss, label = 'Training Loss')
plt.plot(epochs, val_loss, label = 'Validation Loss')
plt.legend(loc = 'upper left')

plt.savefig('./drive/MyDrive/brainmri/loss_model_VGG16.jpg')
plt.show()
plt.close()

The following code is in case we want to continue training for further epochs

In [ ]:
# continue training

filepath = "./drive/MyDrive/brainmri/model_VGG16.h5"
# load the model
new_model = load_model(filepath)
assert_allclose(model.predict(x_train),new_model.predict(x_train),1e-5)

In [ ]:
# # fit the model
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks = [checkpoint]

last_final_epoch = history.epoch[-1] +1

history = new_model.fit(datagen.flow(x_train, y_train, batch_size=32, subset='training'), 
                        validation_data=datagen.flow(x_train, y_train,batch_size=32, subset='validation'),
                        epochs=last_final_epoch + 20, verbose=1,
                        callbacks = callbacks,
                        initial_epoch = last_final_epoch)

In [ ]:
# model.save("./drive/MyDrive/brainmri/VGG16_01_augm.h5")